In [1]:
# now attempt loading ecog data.

import mne
import numpy as np
import pandas as pd
import scipy
import re

# some hyperparameter type things :)
sr = 1000
t = 1000
affix = '_raw'

In [4]:
# load the raw data from .mat
# do preprocessing if necessary (can tweak this part)

fileList = []
subjList = ['bp', 'cc', 'ht', 'jp', 'mv', 'wc', 'zt'] #  'jc' 'wm', <take out

for subj in subjList:
    data = scipy.io.loadmat(f'preprocessing/raw_data/{subj}_fingerflex.mat')['data'].T
    ch = data.shape[0]
    # print(data.shape)
    events = scipy.io.loadmat(f'preprocessing/{subj}_labels.mat')['Labels'][:, 1:3]
    events = np.insert(events, 1, np.zeros(len(events)), axis=1)
    
    features = []
    labels = []

    for event in events:
        time = int(event[0])
        # if sr == 1000: time = int(time/5)
        type = int(event[2])
        features.append([data[i][time:time+1000] for i in range(ch)])
        labels.append(type - 1)
    features = np.array(features)
    labels = np.array(labels)

    # print(features.shape, labels.shape)
    fileList.append((subj, np.array(features), np.array(labels)))


for subj, file, labels in fileList:
    np.save(f'pickles/{subj}_features{affix}', file)
    np.save(f'pickles/{subj}_labels{affix}', labels)

In [2]:
# load .npy files
fileList = []

for subj in ['bp', 'cc', 'ht', 'jp', 'mv', 'wc', 'zt']:
    features = np.load(f'pickles/{subj}_features{affix}.npy')
    labels = np.load(f'pickles/{subj}_labels{affix}.npy')
    fileList.append((subj, features, labels))

In [3]:
#2D CNN w/ Adham's recommendations

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_2D(nn.Module):
    def __init__(self):
        super(ConvNet_2D, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            # nn.Flatten(),
            # lol ok to calculate the size of the linear layer ...
            # width = starting width (time) - combined (kernel sizes + 1) < constant = 1000 - 4 994
            # height = starting height (channels) - combined (kernel heights + 1) < variable = ch - 12
            # nn.Linear(() * 994 * 32, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        x_flat = x.shape[1] * x.shape[2] * x.shape[3]
        flat = nn.Flatten()
        x = flat(x)
        linear = nn.Linear(x_flat, 256)
        # print(x.shape, x_flat)
        x = linear(x)
        x = self.hidden(x)
        return x


In [4]:
# run CNN
# train neural net
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
accuracies = []

for subj, features, labels in fileList:
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    conv_net = ConvNet_2D().to(device)
    criterion = nn.CrossEntropyLoss()
    conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
    # Train the neural network
    for epoch in range(50):
        # print("epoch = ",epoch)
        conv_net.train()
        data = X_train
        targets = y_train
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        # Forward pass
        conv_net_predictions = conv_net(data)
        conv_net_loss = criterion(conv_net_predictions, targets)

        # Backward pass
        conv_net_optimizer.zero_grad()
        conv_net_loss.backward()
        conv_net_optimizer.step()

    # Evaluate the neural network
    conv_net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        data = X_test
        targets = y_test
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)
        conv_net_predictions = conv_net(data)
        _, predicted = torch.max(conv_net_predictions.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    print(f"test accuracy {subj}= ",correct/total)
    accuracies.append(correct/total)

print(f'avg accuracy = {np.mean(accuracies)}')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [5]:
# SVM TIME WOOO
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

avg_acc = 0

for subj, features, labels in fileList:
    #features = csp_features[subj]
    X_train, X_test, y_train, y_test = train_test_split(np.reshape(features, (features.shape[0], -1)), labels, test_size= 0.3) # 70% training 30% test
    
    classifier = BaggingClassifier(svm.SVC(kernel='rbf'), n_estimators=20) # radial basis kernel; er i think they used gamma=0.2 ngl
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    avg_acc += metrics.accuracy_score(y_test, y_pred)

    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

print("Average accuracy: ", avg_acc/len(fileList))

Accuracy bp: 0.46511627906976744
Accuracy cc: 0.6031746031746031
Accuracy ht: 0.47619047619047616
Accuracy jp: 0.5192307692307693


Accuracy mv: 0.5555555555555556
Accuracy wc: 0.5405405405405406
Accuracy zt: 0.4823529411764706
Average accuracy:  0.5203087378483119


In [15]:
# help :P

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


def predict(y_train):
    data = y_train
    pred = []
    with torch.no_grad():
        model.eval()
        data = data.to(device)
        predictions = model(data)
        _, predictions = predictions.max(1)
    return predictions

def evaluate(predictions, targets):
    n_samples, n_correct = 0, 0
    # print(targets.shape, predictions.shape)
    # print(type(targets), targets.shape, type(predictions), len(predictions))
    n_samples += targets.size(0)
    n_correct += (predictions == targets).sum().item()
    acc = n_correct / n_samples
    print(f'accuracy = {acc}')
    return acc

def binary_confusion(y_pred, target):
    # matrix represents [pred 0 targ 0, pred 0 targ 1] [pred 1 targ 0, pred 1 targ 1]
    matrix = [[0, 0],[0, 0]]
    for i in range(y_pred.shape[0]):
        #zero stands for not thumb
        if y_pred[i] == 0 and target[i] == 0:
            matrix[0][0] += 1
        elif y_pred[i] == 0 and target[i] == 1:
            matrix[0][1] += 1
        elif y_pred[i] == 1 and target[i] == 0:
            matrix[1][0] += 1
        elif y_pred[i] == 1 and target[i] == 1:
            matrix[1][1] += 1
    matrix = matrix/np.sum(matrix)
    return matrix

# final NeuralNet

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        nn.Sequential(
            nn.Linear(5, 16),
            nn.Linear(16, 32),
            nn.Linear(32, 5)
        )
        return x
    
# ConvNet_Finger

class ConvNet_Finger(nn.Module):
    def __init__(self):
        super(ConvNet_Finger, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.conv3 = nn.Sequential(
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x

In [19]:
from imblearn.over_sampling import SMOTE

batch_size = 32
dropout = 0.2
n_epochs = 30
learning_rate = 1e-3
weight_decay = 1e-6

torch.cuda.set_device(4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sm = SMOTE()

# train neural net
accuracies = []
for subj, features, labels in fileList:
    ch = features.shape[1]
    input_shape = (-1, 1, ch, t)
    new_features = []
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    for finger in range(5):
        y_train_ = np.where(y_train == finger, 1, 0)
        y_test_ = np.where(y_test == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train_)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))

        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        # Train the neural network

        model.train()
        data = X_res
        targets = y_res
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        for epoch in range(n_epochs):
            # print("epoch = ",epoch)
            model.train()

            # Forward pass
            predictions = model(data)
            loss = criterion(predictions, targets)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}.pt')
        
        pred = predict(torch.as_tensor(X_train.reshape(input_shape), dtype=torch.float32).to(device))
        new_features.append(pred*finger)
    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    
    # final NeuralNet
    model = NeuralNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the neural network
    model.train()
    data = features
    targets = torch.tensor(y_train, dtype=torch.int64).to(device)

    for epoch in range(n_epochs):
        # print("epoch = ",epoch)
        model.train()

        # Forward pass
        predictions = model(data)
        loss = criterion(predictions, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), f'pickles/models/{subj}_final.pt')

    print(f'trained {subj} {finger}')
    new_features = []

    for finger in range(5):
        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))

        y_res = np.where(y_test == finger, 1, 0)
        y_res = torch.tensor(y_res, dtype=torch.int64).to(device)
        pred = predict(torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device))
        print(f'accuracy {subj} finger {finger} = {evaluate(pred, y_res)} {binary_confusion(pred, y_res)})')
        new_features.append(pred*finger)


    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    
    model = NeuralNet().to(device)
    model.load_state_dict(torch.load(f'pickles/models/{subj}_final.pt'))
    acc = evaluate(predict(features), torch.tensor(y_test, dtype=torch.int64).to(device))
    print(f'FINAL accuracy {subj} = {acc} ***************************************')
    accuracies.append(acc)

print(f'avg accuracy = {np.mean(accuracies)}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (206x1081472 and 43456x256)